# scripts to annotate CTCF peaks
since the most likely effect for strong changes in loops should be CTCF altering SNPs, here we just check if the SNPs are near CTCF peaks

In [1]:
import pandas as pd
import numpy as np
import hicstraw 
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from pandarallel import pandarallel
import cooler
import cooltools
import pybedtools as pbed
pandarallel.initialize()
from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns

from functools import reduce

os.makedirs("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/", exist_ok = True)
pbed.helpers.set_tempdir("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/")
bed_genome_file = "/mnt/iusers01/jw01/mdefscs4/hg38.genome"

plt.rcParams['svg.fonttype'] = 'none'

base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

INFO: Pandarallel will run on 28 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
output_dataframe_CD8 = pd.read_csv(f"{base_dir}/HiC_allelic_imbalance/output_dataframe_CD8.csv")
output_dataframe_CD4 = pd.read_csv(f"{base_dir}/HiC_allelic_imbalance/output_dataframe_CD4.csv")

In [3]:
CTCF_peaks_CD4 = pbed.BedTool("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/NEW_references/TF_sites/CTCF/CTCF_CD4_T_CELL_ENCFF148DDI_merged.bed")
CTCF_peaks_CD8 = pbed.BedTool("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/NEW_references/TF_sites/CTCF/CTCF_CD8_T_CELL_ENCFF811QTU_merged.bed")

In [4]:
# create a bedtool with all the SNPs with allelic imbalance
def create_bedtool(dataframe):
    bed_data = dataframe[['chrA', 'rsCoord', 'rsID', 'loopID']].copy()
    bed_data['end'] = bed_data['rsCoord'] + 1
    bed_data.columns = ['chrom', 'start', 'name', 'info','end']
    bedtool = pbed.BedTool.from_dataframe(bed_data[['chrom', 'start', 'end', 'name', 'info']])
    return bedtool

snps_CD4 = create_bedtool(output_dataframe_CD4)
snps_CD8 = create_bedtool(output_dataframe_CD8)

## estimate the number of allelic imbalance loops which contain a SNP in a CTCF peak

In [5]:
snps_CD4_withCTCF = snps_CD4.intersect(CTCF_peaks_CD4, u = True).to_dataframe(header = None, disable_auto_names=True)
snps_CD4_withCTCF.columns = "chrA start end rsID loopID".split()
snps_CD4_withCTCF["CTCF"] = True
snps_CD4_withCTCF = snps_CD4_withCTCF.drop(columns = ["start", "end"])
output_dataframe_CD4 = output_dataframe_CD4.merge(snps_CD4_withCTCF, on = "chrA rsID loopID".split(), how = "left")

In [6]:
sum(output_dataframe_CD4.groupby('loopID')['CTCF'].apply(lambda x: x.isin([True]).any()))/len(output_dataframe_CD4["loopID"].unique())

0.13545042125729098

In [7]:
snps_CD8_withCTCF = snps_CD8.intersect(CTCF_peaks_CD8, u = True).to_dataframe(header = None, disable_auto_names=True)
snps_CD8_withCTCF.columns = "chrA start end rsID loopID".split()
snps_CD8_withCTCF["CTCF"] = True
snps_CD8_withCTCF = snps_CD8_withCTCF.drop(columns = ["start", "end"])
output_dataframe_CD8 = output_dataframe_CD8.merge(snps_CD8_withCTCF, on = "chrA rsID loopID".split(), how = "left")

In [8]:
sum(output_dataframe_CD8.groupby('loopID')['CTCF'].apply(lambda x: x.isin([True]).any()))/len(output_dataframe_CD8["loopID"].unique())

0.15106117353308365

# check with QTL 

In [9]:
ins_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_nominal_CD8_merged.txt", sep = " ")
ins_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_nominal_CD4_merged.txt", sep = " ")
loop_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_nominal_CD8_merged.txt", sep = " ")
loop_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_nominal_CD4_merged.txt", sep = " ")

In [10]:
loop_nominal_CD8 = loop_nominal_CD8[loop_nominal_CD8["nom_pval"] < 0.0001]
loop_nominal_CD4 = loop_nominal_CD4[loop_nominal_CD4["nom_pval"] < 0.0001]
ins_nominal_CD8 = ins_nominal_CD8[ins_nominal_CD8["nom_pval"] < 0.0001]
ins_nominal_CD4 = ins_nominal_CD4[ins_nominal_CD4["nom_pval"] < 0.0001]

In [11]:
def create_bedtool(dataframe):
    # Create a new dataframe with the desired columns
    bed_data = dataframe[['phe_chr', 'var_from', 'var_to', 'var_id', 'phe_id']].copy()
    # Rename columns as needed
    bed_data.columns = ['chrom', 'start', 'end', 'name', 'info']
    # Create a BedTool object from the dataframe
    bedtool = pbed.BedTool.from_dataframe(bed_data)
    return bedtool

snps_ins_nominal_CD8 = create_bedtool(ins_nominal_CD8)
snps_ins_nominal_CD4 = create_bedtool(ins_nominal_CD4)
snps_loop_nominal_CD8 = create_bedtool(loop_nominal_CD8)
snps_loop_nominal_CD4 = create_bedtool(loop_nominal_CD4)

result is number of loops for which one significant snp overlap a CTCF peak vs total number of significant loops

In [12]:
snps_CD4_withCTCF = snps_loop_nominal_CD4.intersect(CTCF_peaks_CD4, u = True).to_dataframe(header = None, disable_auto_names=True)
snps_CD4_withCTCF.columns = "phe_chr start end var_id phe_id".split()
snps_CD4_withCTCF["CTCF"] = True
snps_CD4_withCTCF = snps_CD4_withCTCF.drop(columns = ["start", "end"])
loop_nominal_CD4_with_CTCF = loop_nominal_CD4.merge(snps_CD4_withCTCF, on = "phe_chr var_id phe_id".split(), how = "left")
print(sum(loop_nominal_CD4_with_CTCF.groupby('phe_id')['CTCF'].apply(lambda x: x.isin([True]).any())))
print(len(loop_nominal_CD4_with_CTCF["phe_id"].unique()))

756
11276


In [13]:
snps_CD8_withCTCF = snps_loop_nominal_CD8.intersect(CTCF_peaks_CD8, u = True).to_dataframe(header = None, disable_auto_names=True)
snps_CD8_withCTCF.columns = "phe_chr start end var_id phe_id".split()
snps_CD8_withCTCF["CTCF"] = True
snps_CD8_withCTCF = snps_CD8_withCTCF.drop(columns = ["start", "end"])
loop_nominal_CD8_with_CTCF = loop_nominal_CD8.merge(snps_CD8_withCTCF, on = "phe_chr var_id phe_id".split(), how = "left")
print(sum(loop_nominal_CD8_with_CTCF.groupby('phe_id')['CTCF'].apply(lambda x: x.isin([True]).any())))
print(len(loop_nominal_CD8_with_CTCF["phe_id"].unique()))

936
12205


In [14]:
snps_CD8_withCTCF = snps_ins_nominal_CD8.intersect(CTCF_peaks_CD8, u = True).to_dataframe(header = None, disable_auto_names=True)
snps_CD8_withCTCF.columns = "phe_chr start end var_id phe_id".split()
snps_CD8_withCTCF["CTCF"] = True
snps_CD8_withCTCF = snps_CD8_withCTCF.drop(columns = ["start", "end"])
ins_nominal_CD8_with_CTCF = ins_nominal_CD8.merge(snps_CD8_withCTCF, on = "phe_chr var_id phe_id".split(), how = "left")
print(sum(ins_nominal_CD8_with_CTCF.groupby('phe_id')['CTCF'].apply(lambda x: x.isin([True]).any())))
print(len(ins_nominal_CD8_with_CTCF["phe_id"].unique()))

2470
22934


In [15]:
snps_CD4_withCTCF = snps_ins_nominal_CD4.intersect(CTCF_peaks_CD4, u = True).to_dataframe(header = None, disable_auto_names=True)
snps_CD4_withCTCF.columns = "phe_chr start end var_id phe_id".split()
snps_CD4_withCTCF["CTCF"] = True
snps_CD4_withCTCF = snps_CD4_withCTCF.drop(columns = ["start", "end"])
ins_nominal_CD4_with_CTCF = ins_nominal_CD4.merge(snps_CD4_withCTCF, on = "phe_chr var_id phe_id".split(), how = "left")
print(sum(ins_nominal_CD4_with_CTCF.groupby('phe_id')['CTCF'].apply(lambda x: x.isin([True]).any())))
print(len(ins_nominal_CD4_with_CTCF["phe_id"].unique()))

2039
21439


# temporary tests to be removed

In [16]:
def annotate_CTCF(file, name):
    snp_bed = pbed.BedTool(file)
    snp_bed_withCTCF = snp_bed.intersect(CTCF_peaks_CD8, u = True).to_dataframe(header = None, disable_auto_names=True)
    snp_bed_withCTCF.columns = "chr start end name score".split()
    snp_bed_withCTCF["loci"] = snp_bed_withCTCF["name"].str.split("_").str[-1]
    snp_bed_withCTCF["snp"] = snp_bed_withCTCF["name"].str.split("_").str[0]
    SNPs_withCTCF_loopQTL = snp_bed_withCTCF[snp_bed_withCTCF["snp"].isin(loop_nominal_CD8["var_id"]) | snp_bed_withCTCF["snp"].isin(loop_nominal_CD4["var_id"])]
    display(SNPs_withCTCF_loopQTL)
    return SNPs_withCTCF_loopQTL
annotate_CTCF(f"{base_dir}/metadata/GWAS_files/tsoi2017_LD_0.8_hg38.bed", "psoriasis_tsoi2017")
annotate_CTCF(f"{base_dir}/metadata/GWAS_files/RAmetagwas_all_hg38.ld.bed", "RAmeta")
annotate_CTCF(f"{base_dir}/metadata/GWAS_files/PsA_vs_controls_metagwas_significant.ld.hg38.bed", "PsA_meta")
annotate_CTCF(f"{base_dir}/metadata/GWAS_files/suggestive_snps_hg38_ld.bed", "JIA_suggestive")
annotate_CTCF(f"{base_dir}/metadata/GWAS_files/JIA_credible_snps_hg38.bed", "JIA_credible")
annotate_CTCF(f"{base_dir}/metadata/GWAS_files/elena_hg38.ld.bed", "SSc_elena")

,chr,start,end,name,score,loci,snp


,chr,start,end,name,score,loci,snp
4,chr17,39872866,39872867,rs12936231_rs56750287,0.822789,rs56750287,rs12936231


,chr,start,end,name,score,loci,snp


,chr,start,end,name,score,loci,snp
1,chr13,42478743,42478744,rs2062305_rs12430303,1.000000,rs12430303,rs2062305
3,chr5,177371038,177371039,rs56235845_rs12654812,0.820668,rs12654812,rs56235845


,chr,start,end,name,score,loci,snp
2,chr5,96901621,96901622,rs2287988_rs4869314,0.003514,rs4869314,rs2287988
5,chr13,42478743,42478744,rs2062305_rs12430303,0.040857,rs12430303,rs2062305
8,chr16,28554167,28554168,rs62034351_rs497523,0.008092,rs497523,rs62034351


,chr,start,end,name,score,loci,snp
2,chr17,39754114,39754115,rs2941522_rs883770,0.853170,rs883770,rs2941522
3,chr17,39872866,39872867,rs12936231_rs883770,0.949607,rs883770,rs12936231


,chr,start,end,name,score,loci,snp
2,chr17,39754114,39754115,rs2941522_rs883770,0.853170,rs883770,rs2941522
3,chr17,39872866,39872867,rs12936231_rs883770,0.949607,rs883770,rs12936231


In [44]:
loop_nominal_CD8_with_CTCF[loop_nominal_CD8_with_CTCF["var_id"]=="rs35707106"]

,phe_id,phe_chr,phe_from,phe_to,phe_strd,n_var_in_cis,dist_phe_var,var_id,var_chr,var_from,var_to,nom_pval,r_squared,slope,best_hit,CTCF
66703,35207,chr5,150560001,150820000,+,5214,25815,rs35707106,chr5,150845815,150845815,2.813610e-08,0.456757,1.36125,0,NaN
66893,35238,chr5,150800001,150847500,+,4589,0,rs35707106,chr5,150845815,150845815,2.920850e-05,0.292345,-1.08904,0,NaN


In [ ]:
rs35707106


In [43]:
df = loop_nominal_CD8_with_CTCF[loop_nominal_CD8_with_CTCF["CTCF"].notnull()].sort_values("nom_pval")

In [40]:
import requests
from io import StringIO

def ldtrait(snps, pop="EUR", r2d="r2", r2d_threshold="0.8", win_size="500000", token="df4e68e96257", genome_build="grch38"):
    if not 1 <= len(snps) <= 50:
        raise ValueError("Input should be between 1 to 50 variants.")

    base_url = "https://ldlink.nih.gov/LDlinkRest/ldtrait"
    url_str = f"{base_url}?token={token}"

    snps_to_upload = "\n".join(snps)
    params = {
        "snps": snps_to_upload,
        "pop": pop,
        "r2_d": r2d,
        "r2_d_threshold": r2d_threshold,
        "window": win_size,
        "genome_build": genome_build
    }
    response = requests.post(url_str, json=params)

    if response.status_code == 200:
        data_out = pd.read_csv(StringIO(response.text), sep='\t')
    else:
        raise ValueError(f"Error: {response.status_code}, {response.text}")
    return data_out

def apply_ldtrait_and_merge(long_list):
    def split_list_chunks(input_list, chunk_size=50):
        return [input_list[i:i + chunk_size] for i in range(0, len(input_list), chunk_size)]
    
    list_chunks = split_list_chunks(long_list)
    dataframes = [ldtrait(chunk) for chunk in list_chunks]
    merged_dataframe = pd.concat(dataframes)
    return merged_dataframe

In [41]:
best_snps = loop_nominal_CD8_with_CTCF[loop_nominal_CD8_with_CTCF["CTCF"].notnull()].sort_values("nom_pval").drop_duplicates("phe_id").drop_duplicates("var_id")["var_id"].to_list()
result_dataframe = apply_ldtrait_and_merge(best_snps)
result_dataframe.to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_combined_analysis/allele_specific_hic/calling_allele_specific/ldtrait_CD8_loopQTL.csv")

ProxyError: HTTPSConnectionPool(host='ldlink.nih.gov', port=443): Max retries exceeded with url: /LDlinkRest/ldtrait?token=df4e68e96257 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f531658b550>: Failed to establish a new connection: [Errno 110] Connection timed out')))

In [ ]:
best_snps = loop_nominal_CD4_with_CTCF[loop_nominal_CD4_with_CTCF["CTCF"].notnull()].sort_values("nom_pval").drop_duplicates("phe_id").drop_duplicates("var_id")["var_id"].to_list()
result_dataframe = apply_ldtrait_and_merge(best_snps)
result_dataframe.to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_combined_analysis/allele_specific_hic/calling_allele_specific/ldtrait_CD4_loopQTL.csv")